<a href="https://colab.research.google.com/github/whatmakeart/VFusion3D-colab/blob/main/VFusion3D_Mesh_Generation_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# check if using GPU
!nvidia-smi
# if not using GPU make sure to switch the runtime

In [ ]:
# Clone the Repository
!git clone https://github.com/facebookresearch/vfusion3d
%cd vfusion3d

In [ ]:
# Create the Environment

# Set environment variables using os.environ
import os

os.environ['PYTHON_VERSION'] = '3.8.19'
os.environ['CUDA_VERSION'] = '12.1'


# Install the required packages using pip

!pip install torch==2.3.0+cu121 torchvision==0.18.0+cu121 -f https://download.pytorch.org/whl/torch_stable.html
!pip install transformers
!pip install imageio[ffmpeg]
!pip install --upgrade PyMCubes
!pip install trimesh==4.3.2
!pip install rembg[gpu,cli]
!pip install kiui
!pip install gradio==4.31.4
!pip install gradio-litmodel3d==0.0.1

# Verify installation
import torch
import torchvision
import transformers
import imageio
try:
    import PyMCubes
    print("PyMCubes installed and imported successfully!")
except ImportError:
    print("PyMCubes installation failed.")
import trimesh
import rembg
import kiui
import gradio
import gradio_litmodel3d

print("All packages installed successfully!")


[Google Drive Link](https://drive.google.com/file/d/1b-KKSh9VquJdzmXzZBE4nKbXnbeua42X/view) to model weights checkpoint file

[Hugging Face Link](https://huggingface.co/facebook/vfusion3d/blob/main/model.safetensors) to model weights checkpoint file

Place "vfusion3dckpt" in MyDrive

In [ ]:
# Copy Model Weights from Drive and put it inside ./checkpoints/

from google.colab import drive
drive.mount('/content/drive')
os.makedirs('./checkpoints', exist_ok=True)
!cp /content/drive/MyDrive/vfusion3dckpt ./checkpoints/


In [ ]:
# Make a directory for source image assests and output locations
os.makedirs('./source_images', exist_ok=True)
os.makedirs('./outputs_mesh', exist_ok=True)
os.makedirs('./outputs_video', exist_ok=True)

Place images to batch process in the "source_images" folder

In [ ]:
# Example usages

# Render a mesh
!python -m lrm.inferrer --export_mesh --resume ./checkpoints/vfusion3dckpt --source_path './source_images' --dump_path './outputs_mesh'



In [ ]:
# Render a video
!python -m lrm.inferrer --export_video --resume ./checkpoints/vfusion3dckpt --source_path './source_images' --dump_path './outputs_video'

In [ ]:
# Download the results from the above cell

from google.colab import files
import os

# Directories containing the output files
output_dirs = ['./outputs_mesh', './outputs_video']

# Loop through each directory and download relevant files
for output_dir in output_dirs:
    if os.path.exists(output_dir):
        for filename in os.listdir(output_dir):
            if filename.endswith('.mp4') or filename.endswith('.obj'):  # Check for .mp4 and .obj files
                filepath = os.path.join(output_dir, filename)
                files.download(filepath)
    else:
        print(f"Directory {output_dir} does not exist.")



In [ ]:
# Launch Gradio Visual Interface

import gradio as gr
import imageio
import subprocess
import os

def process_image(image):
    # Save the image temporarily
    os.makedirs('./source_images', exist_ok=True)
    os.makedirs('./outputs_mesh', exist_ok=True)

    image_path = "./source_images/temp.jpg"
    imageio.imwrite(image_path, image)

    # Run the inference command using subprocess
    command = [
        'python', '-m', 'lrm.inferrer', '--export_mesh',
        '--resume', './checkpoints/vfusion3dckpt',
        '--source_path', './source_images',
        '--dump_path', './outputs_mesh'
    ]
    subprocess.run(command, check=True)

    # Return the path to the generated .obj file
    obj_path = "./outputs_mesh/temp.obj"
    if os.path.exists(obj_path):
        return obj_path
    else:
        return None

# Create the Gradio interface
inputs = gr.Image(type="pil", label="Input Image")
outputs = gr.File(label="Download Generated OBJ")

gr.Interface(fn=process_image, inputs=inputs, outputs=outputs, title="VFusion3D OBJ Generator").launch(share=True)
